In [1]:
! eups list -s lsst_distrib

   gdf42428520+f51fc98903 	current w_2023_23 setup


In [10]:
from matplotlib import pyplot as plt
import lsst.daf.butler as daf_butler
import lsst.afw.display as afwDisplay
from lsst.ip.isr.isrTask import IsrTask
from lsst.cp.pipe.cpCombine import CalibCombineTask

from astropy.visualization import imshow_norm, MinMaxInterval,AsinhStretch,LinearStretch,SqrtStretch,ContrastBiasStretch,ZScaleInterval,AsymmetricPercentileInterval,ManualInterval

from lsst.obs.lsst import LsstCam
camera = LsstCam.getCamera()

In [3]:
#repo_path = "/sdf/data/rubin/repo/main"
repo_path = "/sdf/group/rubin/repo/ir2"
collections = 'LSSTCam/raw/all,LSSTCam/calib'.split(",")
butler = daf_butler.Butler(repo_path,collections=collections)
registry = butler.registry

In [4]:
!butler query-dimension-records /sdf/group/rubin/repo/ir2 detector | grep LSSTCam | grep R12

   LSSTCam  45     R12_S00          S00     R12   SCIENCE
   LSSTCam  46     R12_S01          S01     R12   SCIENCE
   LSSTCam  47     R12_S02          S02     R12   SCIENCE
   LSSTCam  48     R12_S10          S10     R12   SCIENCE
   LSSTCam  49     R12_S11          S11     R12   SCIENCE
   LSSTCam  50     R12_S12          S12     R12   SCIENCE
   LSSTCam  51     R12_S20          S20     R12   SCIENCE
   LSSTCam  52     R12_S21          S21     R12   SCIENCE
   LSSTCam  53     R12_S22          S22     R12   SCIENCE


# Get the super flat from Run 13378

In [5]:
repo = '/repo/ir2'
operator = 'lsstccs'
acq_run = '13378'  # This is a D-protocol run.
weekly = 'w_2023_24'

# We use the collection naming scheme to find all of the chained collections for an eo_pipe analysis of 
# the specified run.  For our dataset queries, we'll use these collections.
butler = daf_butler.Butler(repo)
collections_13378 = butler.registry.queryCollections(f"u/{operator}/*_{acq_run}_{weekly}",
                                               collectionTypes=daf_butler.CollectionType.CHAINED)
for item in collections_13378:
    print(item)

u/lsstccs/eo_bias_stability_13378_w_2023_24
u/lsstccs/bias_13378_w_2023_24
u/lsstccs/eo_dark_current_13378_w_2023_24
u/lsstccs/eo_raft_calib_mosaics_13378_w_2023_24
u/lsstccs/dark_13378_w_2023_24
u/lsstccs/eo_eper_13378_w_2023_24
u/lsstccs/eo_ptc_plots_13378_w_2023_24
u/lsstccs/eo_cti_vs_flux_13378_w_2023_24
u/lsstccs/eo_raft_lambda_mosaics_13378_w_2023_24
u/lsstccs/flat_13378_w_2023_24
u/lsstccs/eo_bf_analysis_13378_w_2023_24
u/lsstccs/eo_divisadero_tearing_13378_w_2023_24
u/lsstccs/defects_13378_w_2023_24
u/lsstccs/eo_bright_defects_13378_w_2023_24
u/lsstccs/ptc_13378_w_2023_24
u/lsstccs/eo_read_noise_13378_w_2023_24
u/lsstccs/eo_dark_defects_13378_w_2023_24
u/lsstccs/eo_raft_amp_correlations_13378_w_2023_24
u/lsstccs/eo_persistence_13378_w_2023_24


In [6]:
kwargs={
"datasetType": "cpFlatProc",
"collections": collections_13378,
"where":
"""
instrument='LSSTCam' and
exposure.obs_id in ( 'MC_C_20230619_000202' ) 
"""
}

refs = list(set(registry.queryDatasets(**kwargs)))
exposure_refs = {ref.dataId['detector']: ref for ref in refs}

print( f"You have {len(refs)} in this repository with the condition supplied {kwargs}")

You have 205 in this repository with the condition supplied {'datasetType': 'cpFlatProc', 'collections': ['u/lsstccs/eo_bias_stability_13378_w_2023_24', 'u/lsstccs/bias_13378_w_2023_24', 'u/lsstccs/eo_dark_current_13378_w_2023_24', 'u/lsstccs/eo_raft_calib_mosaics_13378_w_2023_24', 'u/lsstccs/dark_13378_w_2023_24', 'u/lsstccs/eo_eper_13378_w_2023_24', 'u/lsstccs/eo_ptc_plots_13378_w_2023_24', 'u/lsstccs/eo_cti_vs_flux_13378_w_2023_24', 'u/lsstccs/eo_raft_lambda_mosaics_13378_w_2023_24', 'u/lsstccs/flat_13378_w_2023_24', 'u/lsstccs/eo_bf_analysis_13378_w_2023_24', 'u/lsstccs/eo_divisadero_tearing_13378_w_2023_24', 'u/lsstccs/defects_13378_w_2023_24', 'u/lsstccs/eo_bright_defects_13378_w_2023_24', 'u/lsstccs/ptc_13378_w_2023_24', 'u/lsstccs/eo_read_noise_13378_w_2023_24', 'u/lsstccs/eo_dark_defects_13378_w_2023_24', 'u/lsstccs/eo_raft_amp_correlations_13378_w_2023_24', 'u/lsstccs/eo_persistence_13378_w_2023_24'], 'where': "\ninstrument='LSSTCam' and\nexposure.obs_id in ( 'MC_C_20230619_0

In [7]:
exposure_refs.keys()

dict_keys([14, 13, 140, 68, 138, 136, 45, 73, 60, 154, 183, 99, 125, 127, 189, 17, 67, 109, 5, 26, 24, 174, 1, 157, 190, 10, 115, 169, 188, 118, 141, 195, 58, 158, 31, 178, 32, 155, 132, 95, 182, 119, 8, 35, 156, 163, 47, 164, 23, 93, 7, 121, 56, 72, 61, 101, 27, 74, 55, 111, 185, 64, 53, 4, 120, 184, 15, 38, 84, 22, 86, 200, 97, 9, 46, 75, 170, 117, 187, 112, 134, 186, 144, 3, 70, 29, 44, 69, 92, 137, 139, 143, 149, 171, 177, 199, 124, 114, 167, 39, 52, 71, 57, 110, 25, 65, 116, 54, 159, 175, 173, 34, 50, 180, 135, 145, 30, 89, 102, 104, 85, 48, 122, 152, 130, 161, 42, 76, 19, 94, 202, 79, 83, 191, 90, 123, 2, 6, 81, 179, 197, 100, 194, 201, 18, 142, 63, 91, 80, 66, 96, 40, 196, 107, 192, 113, 146, 176, 88, 51, 21, 36, 12, 87, 49, 16, 11, 128, 204, 106, 129, 43, 103, 198, 131, 193, 126, 105, 172, 98, 203, 147, 150, 153, 20, 33, 133, 148, 108, 168, 0, 77, 37, 181, 151, 162, 82, 41, 165, 59, 28, 78, 166, 62, 160])

In [40]:
from lsst.afw.cameraGeom import FOCAL_PLANE, FIELD_ANGLE
adet = camera['R01_S02']
print(f"{adet.getCenter(FIELD_ANGLE) = } (radians)")
print(f"{adet.getCenter(FOCAL_PLANE) = } (mm)")
print(adet.getCorners(FOCAL_PLANE))

adet.getCenter(FIELD_ANGLE) = Point2D(-0.008218561522168856, -0.028726180233102225) (radians)
adet.getCenter(FOCAL_PLANE) = Point2D(-84.760000000000005, -296.25999999999999) (mm)
[Point2D(-105.12, -316.25999999999999), Point2D(-64.400000000000006, -316.25999999999999), Point2D(-64.400000000000006, -276.25999999999999), Point2D(-105.12, -276.25999999999999)]


# make a Mosaic

In [8]:
from  lsst.eo.pipe.plotting import plotting_utils

In [9]:
plotting_utils.make_mosaic(exposure_refs, camera, binSize=8, figsize=(10,10), cmap='plasma', nsig=3.0,
                title='Test Image')

AttributeError: 'DatasetRef' object has no attribute 'get'